<a href="https://colab.research.google.com/github/RafsanJany-44/ARC/blob/master/Primarly_VitalDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# Importing Numpy module
import numpy as np

# Creating a 2-D Numpy array
n_arr = np.array([[45.5, 52, 10],
                  [5, 50, 25]])
print("Given array:")
print(n_arr)

print("\nReplace all elements of array which are greater than 30. to 5.25")
n_arr[0][1] = n_arr[0][0]

print("New array :\n")
print(n_arr)



Given array:
[[45.5 52.  10. ]
 [ 5.  50.  25. ]]

Replace all elements of array which are greater than 30. to 5.25
New array :

[[45.5 45.5 10. ]
 [ 5.  50.  25. ]]


In [2]:
!pip install pyvital vitaldb vital
!pip install neurokit2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.0/55.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.5/202.5 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.0/160.0 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 9.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 824.7/824.7 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 428.8/428.8 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 6.6

In [3]:

import vitaldb
import neurokit2 as nk
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [4]:

Hemodynamic_Parameters = [
'Solar8000/ST_II',
'Solar8000/ST_VI',
'Solar8000/ART_DBP',
'Solar8000/ART_SBP'
]

In [34]:
SRATE = 100
vals = vitaldb.load_case(1, Hemodynamic_Parameters, 1 / SRATE)

In [35]:

MAP = []
indx=1

for i in tqdm(range(vals.shape[0])):

  MAP.append([(vals[i][3] + 2 * vals[i][2])/3,indx])
  if np.isnan(vals[i][0]):
    vals[i][0] = vals[i][1]
  if np.isnan(vals[i][1]):
    vals[i][1] = vals[i][0]
  indx+=1

vals = np.append(vals, MAP ,axis = 1)

  0%|          | 0/1103281 [00:00<?, ?it/s]

In [36]:
vals = vals[~np.isnan(vals).any(axis=1)]

In [37]:
vals.shape

(5299, 6)

In [38]:
vals

array([[-2.00000003e-01, -2.00000003e-01, -8.00000000e+00,
        -8.00000000e+00, -8.00000000e+00,  1.82000000e+02],
       [-2.00000003e-01, -2.00000003e-01, -8.00000000e+00,
        -8.00000000e+00, -8.00000000e+00,  3.82000000e+02],
       [-2.00000003e-01, -2.00000003e-01, -8.00000000e+00,
        -8.00000000e+00, -8.00000000e+00,  5.82000000e+02],
       ...,
       [-1.29999995e+00, -1.29999995e+00,  1.01000000e+02,
         1.98000000e+02,  1.33333333e+02,  1.09068100e+06],
       [-1.29999995e+00, -1.29999995e+00,  1.05000000e+02,
         1.51000000e+02,  1.20333333e+02,  1.09088100e+06],
       [-1.20000005e+00, -1.20000005e+00,  1.05000000e+02,
         1.97000000e+02,  1.35666667e+02,  1.09108100e+06]])

In [7]:
for i in new_vals:
  if np.isnan(i[4]) != True:
    print(i)

Streaming output truncated to the last 5000 lines.
[-3.00000012e-01 -3.00000012e-01  6.00000000e+01  1.30000000e+02
  8.33333333e+01  5.78433000e+05]
[-3.00000012e-01 -3.00000012e-01  5.90000000e+01  1.27000000e+02
  8.16666667e+01  5.78633000e+05]
[-3.00000012e-01 -3.00000012e-01  6.00000000e+01  1.29000000e+02
  8.30000000e+01  5.78833000e+05]
[-3.00000012e-01 -3.00000012e-01  6.00000000e+01  1.27000000e+02
  8.23333333e+01  5.79033000e+05]
[-3.00000012e-01 -3.00000012e-01  6.10000000e+01  1.27000000e+02
  8.30000000e+01  5.79233000e+05]
[-3.00000012e-01 -3.00000012e-01  6.00000000e+01  1.29000000e+02
  8.30000000e+01  5.79433000e+05]
[-3.00000012e-01 -3.00000012e-01  5.90000000e+01  1.27000000e+02
  8.16666667e+01  5.79633000e+05]
[-3.00000012e-01 -3.00000012e-01  5.90000000e+01  1.27000000e+02
  8.16666667e+01  5.79833000e+05]
[-5.00000000e-01 -5.00000000e-01  6.00000000e+01  1.27000000e+02
  8.23333333e+01  5.80034000e+05]
[-5.00000e-01 -5.00000e-01  6.10000e+01  1.27000e+02  8.30

In [8]:
for i in range(new_vals.shape[0]):
  if np.isnan(new_vals[i][0]) != True:
    print(new_vals[i])

Streaming output truncated to the last 5000 lines.
             nan  6.29438000e+05]
[-4.00000006e-01 -4.00000006e-01  5.60000000e+01  1.15000000e+02
  7.56666667e+01  6.29637000e+05]
[-4.00000006e-01 -4.00000006e-01  5.60000000e+01  1.14000000e+02
  7.53333333e+01  6.29837000e+05]
[-3.00000012e-01 -3.00000012e-01  5.60000000e+01  1.17000000e+02
  7.63333333e+01  6.30037000e+05]
[-3.00000012e-01 -3.00000012e-01  5.50000000e+01  1.14000000e+02
  7.46666667e+01  6.30239000e+05]
[-3.00000012e-01 -3.00000012e-01  5.70000000e+01  1.17000000e+02
  7.70000000e+01  6.30439000e+05]
[-3.00000012e-01 -3.00000012e-01  5.60000000e+01  1.15000000e+02
  7.56666667e+01  6.30638000e+05]
[-3.00000012e-01 -3.00000012e-01  5.70000000e+01  1.15000000e+02
  7.63333333e+01  6.30838000e+05]
[-3.00000012e-01 -3.00000012e-01  5.60000000e+01  1.16000000e+02
  7.60000000e+01  6.31038000e+05]
[-3.00000012e-01 -3.00000012e-01  5.60000000e+01  1.15000000e+02
  7.56666667e+01  6.31238000e+05]
[-4.00000006e-01 -4.0000

In [9]:
import pandas as pd

df = pd.DataFrame(new_vals)
df.columns = [
'Solar8000/ST_II',
'Solar8000/ST_VI',
'Solar8000/ART_DBP',
'Solar8000/ART_SBP',
'MAP',
'2_SEC_Time'
]


In [10]:
df.to_csv("Patient_1.csv",index=False)

In [11]:
df.shape

(1103281, 6)

In [12]:
df.head()

,Solar8000/ST_II,Solar8000/ST_VI,Solar8000/ART_DBP,Solar8000/ART_SBP,MAP,2_SEC_Time
0,NaN,NaN,NaN,NaN,NaN,1.0
1,NaN,NaN,NaN,NaN,NaN,2.0
2,NaN,NaN,NaN,NaN,NaN,3.0
3,NaN,NaN,NaN,NaN,NaN,4.0
4,NaN,NaN,NaN,NaN,NaN,5.0


In [13]:

df = df.query("MAP.notna()")

In [14]:
df_trks = pd.read_csv('https://api.vitaldb.net/trks')  # read track list
df_cases = pd.read_csv("https://api.vitaldb.net/cases")  # read case information


In [15]:
list(df_trks['tname'])

['BIS/BIS',
 'BIS/EEG1_WAV',
 'BIS/EEG2_WAV',
 'BIS/EMG',
 'BIS/SEF',
 'BIS/SQI',
 'BIS/SR',
 'BIS/TOTPOW',
 'Primus/AWP',
 'Primus/CO2',
 'Primus/COMPLIANCE',
 'Primus/ETCO2',
 'Primus/EXP_DES',
 'Primus/EXP_SEVO',
 'Primus/FEN2O',
 'Primus/FEO2',
 'Primus/FIN2O',
 'Primus/FIO2',
 'Primus/INCO2',
 'Primus/INSP_DES',
 'Primus/INSP_SEVO',
 'Primus/MAC',
 'Primus/MAWP_MBAR',
 'Primus/MV',
 'Primus/PAMB_MBAR',
 'Primus/PEEP_MBAR',
 'Primus/PIP_MBAR',
 'Primus/PPLAT_MBAR',
 'Primus/RR_CO2',
 'Primus/SET_AGE',
 'Primus/SET_FIO2',
 'Primus/SET_FRESH_FLOW',
 'Primus/SET_INSP_PAUSE',
 'Primus/SET_INSP_TM',
 'Primus/SET_INTER_PEEP',
 'Primus/SET_PIP',
 'Primus/SET_RR_IPPV',
 'Primus/SET_TV_L',
 'Primus/TV',
 'Primus/VENT_LEAK',
 'SNUADC/ART',
 'SNUADC/ECG_II',
 'SNUADC/ECG_V5',
 'SNUADC/PLETH',
 'Solar8000/ART_DBP',
 'Solar8000/ART_MBP',
 'Solar8000/ART_SBP',
 'Solar8000/BT',
 'Solar8000/ETCO2',
 'Solar8000/FEO2',
 'Solar8000/FIO2',
 'Solar8000/GAS2_EXPIRED',
 'Solar8000/GAS2_INSPIRED',
 'Solar